## WATEM::WhastApp

### Imports

In [1]:
from wabs import *
import xbuilder as xb
import wa_actions as wact
import wa_main as wm

xx = WABS()
dd = xx.get_driver

[WDM] - Downloading:  95%|█████████████████████████████████████████████████████▎  | 6.46M/6.79M [00:06<00:00, 1.27MB/s]

(4) WhatsApp


### Decorator & Closer for general use

In [14]:
from functools import wraps
import subprocess

def outer_function(arg):
    def debug(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print('function Name: ', func.__name__)
            print('Running from: ', func.__module__)
            for n, i in enumerate(args):
                print('arbitary arg >> number:',n ,'| value:', i, '|', type(i))
            for k, n in kwargs.items():
                print('keyword arg: name:', k, '| value: ', n, '|', type(i))
            result = func(*args, **kwargs)
            print('Result:', result)
            return arg + str(result)
        return wrapper
    return debug

#@outer_function('calculated result: ')
#def av(a1,a2):
#    print('av result ', a1*a2)
#    return a1*a2
#q = av(2,3)
#print(q)

def totxt(fp:str, data:str, method='a+'):
    with open(fp, method) as f:
        f.write(data)

def tex(f):
    @wraps(f)
    def te(*a, **k):
        try: return f(*a, **k)
        except: return 0
    return te

def closr(f):
    def inn(*a, **k):
        return f(*a, **k)
    return inn

def subp_execute(cmd):
    command = cmd.split(' ') if type(cmd) is not list and ' ' in cmd else cmd
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return str(process.stdout.read())

function Name:  av
Running from:  __main__
arbitary arg >> number: 0 | value: 2 | <class 'int'>
arbitary arg >> number: 1 | value: 3 | <class 'int'>
av result  6
Result: 6
calculated result: 6


### Selenium Required Functions

In [11]:
def sidepane_scroll(d, initial = 1, scrolls=50):
    for i in range(1, scrolls):
        d.execute_script("document.getElementById('pane-side').scrollTop={}".format(initial))
        initial = initial + i + 1
    return initial

def msg_scoll_up(d):
    d.find_element(By.XPATH, "//div[@class='_5kRIK']").send_keys(Keys.HOME)
    tm.sleep(2)
    
def wtx(d, xpt, wt=1):
    try: return d.find_element(By.XPATH, xpt).text
    except: return None

def ww(d, xpt, sec=2):
    wait = WebDriverWait(d, sec)
    try: return wait.until(EC.presence_of_element_located((By.XPATH, xpt)))
    except: return None
    
def wx(d, xpt, elements=None, wait=None, text=None, click=None):
    try:
        if elements is not None:
            return d.find_elements(By.XPATH, xpt)
        elif wait is not none: 
            y = ww(d, xpt, wait)   
        else:
            y = d.find_element(By.XPATH, xpt)
        if text is not None: return y.text
        elif click is not None: 
            y.click()
            return 1
        else: return y
    except: return None

def xpbulk(d, xpt):
    try:
        if type(xpt) is str: return wtx(d, xpt)
        elif type(xpt) is list: return [wtx(d, xpt) for n in xpt]
        elif type(xpt) is dict:
            return {k : [wtx(d, xpt) for n in v] if type(v) is list else wtx(d, xpt) for k, v in xpt.items()}
        else: return None
    except:
        return None
        
def write_msg(d, xpt, text):
    ac = ActionChains(d)
    ac.move_to_element(ww(d,xpt)).click().perform()
    if chr(10) not in text:
        ac.send_keys(text).send_keys(Keys.RETURN).perform()
    else:
        xx = text.split(chr(10))
        for line in xx:
            ac.send_keys(line).perform()
            ac.key_down(Keys.SHIFT).key_down(Keys.ENTER).key_up(Keys.SHIFT).key_up(Keys.ENTER).perform()
        ac.send_keys(Keys.RETURN).perform()
    
def action(d, k, v):
    ac = ActionChains(d)
    if '_sendtxt_' in k: write_msg(d, v[0], v[1])
    elif 'key_to_element': ac.send_keys_to_element(ww(d,v[0]), v[1]).send_keys(Keys.RETURN).perform()
    elif '_clk_' in k: ac.click(ww(d, v[0])).perform()
    elif '_justmove_' in k: ac.move_to_element(ww(d, v[0])).perform()
    elif '_hoverclick_' in k: 
        ac.move_to_element(ww(d, v[0])).pause(.5).perform()
        ac.click(ww(v[1])).perform()
    elif '_moveclick_' in k: ac.move_to_element(ww(d, v[0])).pause(1).click().perform()
    else: return None

### General Python Self Built Method

In [12]:
def update_dict_values(a, b):
    '''updating key & values using append then set procedure'''
    c = {}
    for k, v in a.items():
        if k in list(b):
            c[k] = list(set(a[k] + b[k]))
        else:
            c[k] = a[k]
    return {**b, **c}

def update_key_value(a, b):
    '''updating key & values (values will be appened for existing key)'''
    c = {}
    if len(a) == 0: return b
    tls = lambda ls : ls if type(ls) is list else [ls]
    for k, v in a.items():
        c[k] = tls(a[k]) + tls(b[k]) if k in list(b) else tls(a[k])
    else: return {**b, **c}
    
def value_already_exist(dc, ky, val):
    try:
        a = dc.get(ky, val)
        return True
    except: return false
    
def dict_to_df(self, dc, out=None):
    x = dt.datetime.now().strftime('%b%d%Y_%H%M')
    opt = x  + '.csv' if out is None else x + '_' + out + '.csv'
    df = pd.DataFrame.from_dict(dc).to_csv(opt)
    return df

### WhatsApp Specific Functions

In [13]:
msgbase = "(//div[@id='main']//div[@data-testid='msg-container'])"
msgbs = "//div[@class='ItfyB _3nbHh']"

def search_msg_bytext(text = [('@class','_21Ahp'), 'omi'], index=None):
    txtm = '(' + contains_gen(contains, text,'div') 
    txtm = txtm + "[" + str(index) + "]" if type(index) is not None else txtm
    fxpt = txtm + "/ancestor::div[@data-testid='msg-container']"
    return fxpt

def search_msg_by_sender_text(sender='omi', text=None, index=None):
    txtm = "(//div[@class='ItfyB _3nbHh'])[" + starts_with_no_attr(sender) + ']'
    axes="/ancestor::div[@data-testid='msg-container']"
    if text is not None:
        for n in text:
            nd = tuple(n) if n is str else n
            txtm = contains_add(contains, txtm, nd , 'starts-with')
    xptx = txtm + '[' + str(index) + ']'  + axes if index is not None else txtm + axes
    return xptx

pnbase ="(//div[@id='pane-side']//div[@data-testid='cell-frame-container'])"
pnbs = "//div[@class='_8nE1Y']"
pane_chat_byname = lambda chatname: "//div[@class='y_sn4']//span" + "[contains(.,'" + chatname + "')]//ancestor::" + pnbase
pane_chat_unread_byname = lambda chatname : pane_chat_byname(chatname) + "//span[@data-testid='icon-unread-count']"

def wapane_chat_type(d, bs):
    xpt = bs + "//div[@data-testid='chatlist-status-v3-ring']"
    if ww(d, xpt.strip(' ')) is None: return 'group'
    else: return 'contact'
    
def chatinfo(d, n=None, pane_base="//div[@id='pane-side']//div[@data-testid='cell-frame-container']"):
    base = '(' + pane_base + "//div[@class='_8nE1Y'])" if '_8nE1Y' not in pane_base else pane_base
    bi = base + '[' + str(n) + ']' if n is not None else base
    bi4ctype = '(' + pane_base + ')[' + str(n) + ']' if n is not None else pane_base 
    if ww(d, bi) is None: return None
    print('pane coll xpt not none: ', bi)
    dc = {'chat_type': [wapane_chat_type(d, bi4ctype)],
        'chat_name' : [bi + "//div[@class='y_sn4']//span"],
        'last_text' : [bi + "//div[@class='vQ0w7']//span[@dir='ltr']"],
        'last_sender' : [bi + "//div[@class='vQ0w7']//span[@dir='auto']"],
        'time' : [bi + "/div[@class='y_sn4']//div[@class='Dvjym']"],
        'new_msg_notif' : [bi + "//span[@data-testid='icon-unread-count']"]}
    for k, v in dc.items():
        v = [v] if type(v) is str else v
        dc[k] = [wtx(d, j) for j in v]
    return dc

def msg_sender_dttm(d, xpt):
    elm = ww(d, xpt) if type(xpt) is str else xpt
    html = elm.get_attribute('innerHTML')
    sp = BeautifulSoup(html, "lxml")
    x = sp.find_all('div')
    for i in x:
        try:
            st = i['data-pre-plain-text']
            break
        except:pass
    else: return None, None
    try:
        split_point = st.find(']')
        sender = st[split_point+1:].replace(':','').strip(' ')
        dttm = parse(st.upper()[1:split_point])
        return dttm.strftime("%Y-%m-%d %H:%M"), sender
    except: None, None

def msginfo(d, n=None, base="(//div[@id='main']//div[@data-testid='msg-container'])"):
    bi = base + '[' + str(n) + ']' if n is not None else base
    print('msgbase is being collecting:', bi)
    elm = ww(d, bi)
    if elm is None: return None
    else:
        try: ActionChains(d).move_to_element(ww(d, bi + "//div[@role='button'][contains(text(),'Read more')]")).click().perform()
        except: pass
        dttm, sender = msg_sender_dttm(d, elm)
        dc = {'sender' : [sender],
              'text' : [wtx(d, bi + "//div[@class='_21Ahp']/span[1]/span")],
              'q_sender' : [wtx(d, bi + "//div[@class='_3pMOs yKTUI']//div[1]/span")],
              'q_text' : [wtx(d, bi + "//div[@class='_3pMOs yKTUI']//div[2]/span")],
              'datetime' : [dttm],
              'sender_visible': [wtx(d, bi + "//div[@class='_27K43 _31p5Q']/div[1]//span[@dir='auto']")],
              'time': [wtx(d, bi + "//div[@data-testid='msg-meta']/span")]
              }
        dc['img'] = 'image' if ww(d, bi + "//div[@data-testid='image-thumb']", 1) is not None else 'NoImage'
        return dc
    


IndentationError: expected an indented block (2586682795.py, line 28)

### The Perfect Iterator

In [ ]:
def search_in(d, func, scroll_func, scroll_arg=None, xpt=None, last=False, chat=None):
    if chat is not None: 
        if wact.select_chat(d, chat) == 0: return "none"
    dc, i, scroll_limit = defaultdict(list), 1, 0
    while scroll_limit<10:
        if last==True: xp = func(d,'last()-'+str(i),xpt) if xpt is not None else func(d,'last()-'+str(i))
        else: xp = func(d,i,xpt) if xpt is not None else func(d,i)
        if xp is not None:
            yield xp, i, scroll_limit
            i, scroll_limit = i + 1, 0
        else:
            if scroll_limit>1:
                scroll_arg = scroll_func(d, scroll_arg) if scroll_arg is not None else scroll_func(d)
                i = 1 if scroll_arg is not None else i
            scroll_limit = scroll_limit + 1

msg = search_in(d=dd, func=wm.msginfo, xpt="(//div[@id='main']//div[@data-testid='msg-container'])", 
               scroll_func=wm.just_move, scroll_arg=None, chat='Emergency')

pane = search_in(d=dd, func=wm.side_pane, scroll_func=wm.sidepane_scroll, scroll_arg=1)

In [ ]:
for i in ab:
    dc, inx, scroll = i
    print(dc)
    if inx > 20: break
print('nice job')